In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.dates as mdates
from sklearn.preprocessing import MinMaxScaler
import math
from minisom import MiniSom
import numpy as np
from tslearn.metrics import soft_dtw
from tslearn.barycenters import dtw_barycenter_averaging
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
%matplotlib inline

In [43]:
cr_score = pd.read_csv('SOM병합_CRscore.csv', encoding='euc-kr')

In [66]:
len(cr_score)

3847

In [44]:
cr_score['인증'] = cr_score['인증'].fillna('[]')
cr_score['특허'] = cr_score['특허'].fillna('[]')

In [45]:
cr_score

,Unnamed: 0,사업자번호,Cluster,기업코드,최종연구개발점수,최종연구개발등급,연구개발투자점수,인적자원구성원 등급,연구개발기술지원점수,인적자원복지등급,...,법률적 책임 등급,윤리적 책임 점수,윤리적 책임 등급,자선적 책임 점수,자선적 책임 등급,재무,상품,산업군,특허,인증
0,0,1.478700e+09,Cluster 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
1,1,1.198680e+09,Cluster 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
2,2,5.838600e+09,Cluster 1,7634.0,60.0,D,30.0,D,18.0,D,...,A,100.0,A,50.0,D,결산년도 총자산 자본금 자본총계 매출...,"에어로젤 단열커버, 서보모터솔루션 등 개발",응용소프트웨어 개발 및 공급업,[],[]
3,3,1.148701e+09,Cluster 1,7546.0,60.0,D,30.0,D,18.0,D,...,A,100.0,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,정보없음,유선통신업,[],[]
4,4,1.378613e+09,Cluster 1,7533.0,64.5,D,30.0,D,22.5,C,...,A,100.0,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,전력 수/배전반 외,배전반 및 전기자동제어반 제조업,[],"벤처기업확인,2015', '이노비즈인증,2015'"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3842,3842,6.091425e+09,Cluster 9,4576.0,67.5,D,30.0,D,22.5,C,...,A,100.0,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,"홈페이제작,SW개발",컴퓨터시스템 통합 자문 및 구축 서비스업,[],[]
3843,3843,1.208799e+09,Cluster 9,6361.0,60.0,D,30.0,D,18.0,D,...,A,100.0,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,"소프트웨어, 보안, 백업 프로그램","컴퓨터 및 주변장치, 소프트웨어 도매업",[],[]
3844,3844,3.038144e+09,Cluster 9,8819.0,64.5,D,30.0,D,22.5,C,...,A,100.0,A,50.0,D,결산년도 총자산 자본금 자본총계 매...,서버,"컴퓨터 및 주변장치, 소프트웨어 소매업",[],[]
3845,3845,1.378615e+09,Cluster 9,2221.0,60.0,D,30.0,D,18.0,D,...,A,100.0,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,홈페이지 제작,컴퓨터 프로그래밍 서비스업,[],"['벤처기업확인,2013']"


In [46]:
cluster_innobiz = cr_score[cr_score['인증'].values != 0]
cluster_innobiz.reset_index(drop=True, inplace=True)

In [47]:
innobiz_year = []
innobiz_code = []
innobiz_yr = []
innobiz_cd = []
innobiz_df = pd.DataFrame()

cluster_innobiz['인증'] = cluster_innobiz['인증'].replace("''",'')

for m in range(len(cluster_innobiz)):
    innobiz_list = cluster_innobiz['인증'][m].split(',')
    innobiz_year = []
    innobiz_code = []
    for i in range(0,len(innobiz_list),2):
        if innobiz_list[i] == '이노비즈인증':
            innobiz_year.append('이노비즈')
            innobiz_df = pd.concat([innobiz_df, pd.DataFrame(cluster_innobiz.iloc[m,:]).transpose()], axis=0)
    innobiz_yr.append(innobiz_year)

In [48]:
tmp = pd.DataFrame(innobiz_yr)
tmp.columns = ['이노비즈']

In [49]:
cr_score = pd.concat([cr_score, tmp], axis=1)

In [50]:
cr_score

,Unnamed: 0,사업자번호,Cluster,기업코드,최종연구개발점수,최종연구개발등급,연구개발투자점수,인적자원구성원 등급,연구개발기술지원점수,인적자원복지등급,...,윤리적 책임 점수,윤리적 책임 등급,자선적 책임 점수,자선적 책임 등급,재무,상품,산업군,특허,인증,이노비즈
0,0,1.478700e+09,Cluster 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[],None
1,1,1.198680e+09,Cluster 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[],None
2,2,5.838600e+09,Cluster 1,7634.0,60.0,D,30.0,D,18.0,D,...,100.0,A,50.0,D,결산년도 총자산 자본금 자본총계 매출...,"에어로젤 단열커버, 서보모터솔루션 등 개발",응용소프트웨어 개발 및 공급업,[],[],None
3,3,1.148701e+09,Cluster 1,7546.0,60.0,D,30.0,D,18.0,D,...,100.0,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,정보없음,유선통신업,[],[],None
4,4,1.378613e+09,Cluster 1,7533.0,64.5,D,30.0,D,22.5,C,...,100.0,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,전력 수/배전반 외,배전반 및 전기자동제어반 제조업,[],"벤처기업확인,2015', '이노비즈인증,2015'",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3842,3842,6.091425e+09,Cluster 9,4576.0,67.5,D,30.0,D,22.5,C,...,100.0,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,"홈페이제작,SW개발",컴퓨터시스템 통합 자문 및 구축 서비스업,[],[],None
3843,3843,1.208799e+09,Cluster 9,6361.0,60.0,D,30.0,D,18.0,D,...,100.0,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,"소프트웨어, 보안, 백업 프로그램","컴퓨터 및 주변장치, 소프트웨어 도매업",[],[],None
3844,3844,3.038144e+09,Cluster 9,8819.0,64.5,D,30.0,D,22.5,C,...,100.0,A,50.0,D,결산년도 총자산 자본금 자본총계 매...,서버,"컴퓨터 및 주변장치, 소프트웨어 소매업",[],[],None
3845,3845,1.378615e+09,Cluster 9,2221.0,60.0,D,30.0,D,18.0,D,...,100.0,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,홈페이지 제작,컴퓨터 프로그래밍 서비스업,[],"['벤처기업확인,2013']",None


In [52]:
increase_cluster = ['Cluster 2','Cluster 20','Cluster 32','Cluster 51', 'Cluster 56','Cluster 58']
decrease_cluster = ['Cluster 6' , 'Cluster 24','Cluster 54']

cr_score['추세'] = 0

for i in range(len(cr_score)):
    if cr_score.iloc[i,2] in increase_cluster:
        cr_score.iloc[i,-1] = '증가'
    elif cr_score.iloc[i,2] in decrease_cluster:
        cr_score.iloc[i,-1] = '감소'

In [53]:
cr_increase = cr_score[cr_score['추세']=='증가']
cr_decrease = cr_score[cr_score['추세']=='감소']

In [58]:
len(cr_increase[cr_increase['이노비즈']=='이노비즈'])

11

In [59]:
len(cr_increase)

742

In [56]:
len(cr_increase[cr_increase['이노비즈']=='이노비즈']) / len(cr_increase) * 100

1.482479784366577

In [60]:
len(cr_decrease[cr_decrease['이노비즈']=='이노비즈']) / len(cr_decrease) * 100

3.3457249070631967

In [61]:
cr_innobiz = cr_score[cr_score['이노비즈']=='이노비즈']
cr_innobiz

,Unnamed: 0,사업자번호,Cluster,기업코드,최종연구개발점수,최종연구개발등급,연구개발투자점수,인적자원구성원 등급,연구개발기술지원점수,인적자원복지등급,...,윤리적 책임 등급,자선적 책임 점수,자선적 책임 등급,재무,상품,산업군,특허,인증,이노비즈,추세
128,128,1.388173e+09,Cluster 10,5339.0,79.5,C,42.0,B,22.5,C,...,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,"하수관거 유지관리 모니터링 시스템, 상수도 통합 관리시스템",컴퓨터시스템 통합 자문 및 구축 서비스업,"19-A-1965,2020', '19-A-1965,2020', '19-C-0592,...","이노비즈인증,2018', '벤처기업확인,2012'",이노비즈,0
186,186,1.178156e+09,Cluster 11,6756.0,60.0,D,30.0,D,18.0,D,...,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,웹사이트 구축 및 운영 외,시스템 소프트웨어 개발 및 공급업,[],"이노비즈인증,2017', '벤처기업확인,2016'",이노비즈,0
218,218,1.218624e+09,Cluster 11,2627.0,70.5,C,36.0,C,22.5,C,...,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,Logpresso,응용소프트웨어 개발 및 공급업,"16-0271,2016'","이노비즈인증,2020', '벤처기업확인,2019'",이노비즈,0
232,232,5.148199e+09,Cluster 12,3963.0,66.5,D,30.0,D,22.5,C,...,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,"인공지능, 임베디드 SW, 솔루션",응용소프트웨어 개발 및 공급업,[],"이노비즈인증,2020', '벤처기업확인,2020'",이노비즈,0
271,271,2.208650e+09,Cluster 13,3148.0,67.5,D,30.0,D,22.5,C,...,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,"IoT(LBS, RF 및 CCTV)관련 솔루션 외",시스템 소프트웨어 개발 및 공급업,[],"이노비즈인증,2021', '벤처기업확인,2019', '이노비즈인증,2019'",이노비즈,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3509,3509,2.658700e+09,Cluster 60,6980.0,60.0,D,30.0,D,18.0,D,...,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,"IT 컨설팅, CRM시스템구축, RPA개발 외",응용소프트웨어 개발 및 공급업,[],"이노비즈인증,2021'",이노비즈,0
3560,3560,1.198604e+09,Cluster 62,5060.0,83.5,B,45.0,A,22.5,C,...,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,"통합관제, 물리보안 솔루션",응용소프트웨어 개발 및 공급업,"AC-07348,2019', 'AC-07348,2019', 'HS19OV010P-U...","이노비즈인증,2021', '벤처기업확인,2020'",이노비즈,0
3615,3615,4.168187e+09,Cluster 63,7550.0,67.5,D,33.0,D,22.5,C,...,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,"부가통신, 정보통신공사",내부 통신배선 공사업,"RAQ1395,2014'","이노비즈인증,2015', '벤처기업확인,2015'",이노비즈,0
3660,3660,1.178175e+09,Cluster 64,5878.0,65.5,D,30.0,D,22.5,C,...,A,50.0,D,결산년도 총자산 자본금 자본총계 ...,"ERP, MES, ESL 등",응용소프트웨어 개발 및 공급업,[],"이노비즈인증,2020'",이노비즈,0


In [63]:
cr_innobiz['추세'] = 0

for i in range(len(cr_innobiz)):
    if cr_innobiz.iloc[i,2] in increase_cluster:
        cr_innobiz.iloc[i,-1] = '증가'
    elif cr_innobiz.iloc[i,2] in decrease_cluster:
        cr_innobiz.iloc[i,-1] = '감소'

<ipython-input-63-b2016470b7f6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cr_innobiz['추세'] = 0


In [65]:
cr_innobiz['추세'].value_counts()

0     49
증가    11
감소     9
Name: 추세, dtype: int64